In [5]:
# import libraries

In [6]:
# investigate data

In [19]:
import sqlite3
import pandas as pd


def to_csv():
    db = sqlite3.connect(r'..\data\northWind.sqlite')
    cursor = db.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    errors_list = []
    for table_name in tables:
        table_name = table_name[0]
        
        this_query = "SELECT * from %s" % table_name
        
        table = None
        try:
            table = pd.read_sql_query(this_query, db)
        except:
            errors_list.append("failed to execute query {" + this_query + "}")
            
        if isinstance(table, pd.core.frame.DataFrame):
            csvPath = "".join(["..", "\\", "data", "\\", table_name, ".csv"])
            
            wrote_csv = False
            try:
                table.to_csv(csvPath, index_label='index')
                wrote_csv = True
            except:
                pass

            if wrote_csv:
                print("wrote table {" + table_name + "} to file {" + csvPath + "}.")
            else:
                errors_list.append("failed to write table {" + table_name + "} to file {" + csvPath + "}")
                
    cursor.close()
    db.close()
    return errors_list

print(to_csv())

wrote table {Employee} to file {..\data\Employee.csv}.
wrote table {Category} to file {..\data\Category.csv}.
wrote table {Customer} to file {..\data\Customer.csv}.
wrote table {Shipper} to file {..\data\Shipper.csv}.
wrote table {Supplier} to file {..\data\Supplier.csv}.
wrote table {Product} to file {..\data\Product.csv}.
wrote table {OrderDetail} to file {..\data\OrderDetail.csv}.
wrote table {CustomerCustomerDemo} to file {..\data\CustomerCustomerDemo.csv}.
wrote table {CustomerDemographic} to file {..\data\CustomerDemographic.csv}.
wrote table {Region} to file {..\data\Region.csv}.
wrote table {Territory} to file {..\data\Territory.csv}.
wrote table {EmployeeTerritory} to file {..\data\EmployeeTerritory.csv}.
['FAILED to execute query {SELECT * from Order}']


# Question 1: Is there a difference in quantity of products sold by discount rate? 

In [2]:
# obtain necessary data

## {Put test name here}
H0: 

HA:

In [4]:
# perform test

## Conclusion

## EffectSize/PowerAnalysis/Tukey Testing (if needed)

# Question 2: Is there a difference in number of orders and territory? 

In [2]:
# obtain necessary data

## {Put test name here}
H0: 

HA:

In [4]:
# perform test

## Conclusion

# Question 3: {You decide}

In [2]:
# obtain necessary data

## {Put test name here}
H0: 

HA:

In [4]:
# perform test

## Conclusion

## EffectSize/PowerAnalysis/Tukey Testing (if needed)